In [ ]:
import os
import re
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import logging
logger = logging.getLogger(__name__)

box_color = '#86afb8'

In [ ]:
def load_data(directory):
    data = []
    columns = ['schedule_type', 'n_drones', 'sigma', 'W', 'time_limit', 'execution_time', 'iteration', 'sim_timestamp', 'solve_time', 'r_charge', 'r_deplete']

    for subdir in os.listdir(directory):
        if os.path.isdir(os.path.join(basedir, subdir)):
            match = re.match("milp_ndrones(\d+)_sigma(\d+)_W(\d+)_tl(\d+)_rc(0.\d+)_rd(0.\d+)", subdir)
            if match:
                # MILP simulation
                schedule_type = "milp"
                n_drones, sigma, W, time_limit, r_charge, r_deplete = int(match[1]), int(match[2]), int(match[3]), int(match[4]), float(match[5]), float(match[6])
            match = re.match("naive_(\d+)_rc(0.\d+)_rd(0.\d+)", subdir)
            if match:
                # Naive simulation
                schedule_type = "naive"
                n_drones, sigma, W, time_limit, r_charge, r_deplete = int(match[1]), None, None, None, float(match[2]), float(match[3])
 
            if not schedule_type:
                logger.debug(f"non-experiment directory, skipping '{subdir}'")

            # execution time
            fname = os.path.join(basedir, subdir, "execution_time.txt")
            try:
                with open(fname, 'r') as f:
                    execution_time = float(f.read())
            except FileNotFoundError:
                logger.debug("failed to read execution time")
                execution_time = np.NaN

 
            # solve times
            fname = os.path.join(basedir, subdir, "solve_times.csv")
            df_solve_times = pd.read_csv(fname)            
            for _, row in df_solve_times.iterrows():
                it, sim_timestamp, solve_time = row[['iteration', 'sim_timestamp', 'solve_time']]
                data.append([schedule_type, n_drones, sigma, W, time_limit, execution_time, it, sim_timestamp, solve_time, r_charge, r_deplete])    
    res = pd.DataFrame(data=data, columns=columns).fillna(0)
    res['W'] = res['W'].astype(int)
    return res

# Planning horizon

## Increase $\sigma$

In [ ]:
basedir = "../out/villalvernia/1_planning_horizon/increase_sigma"
df_increase_sigma = load_data(basedir)
df_increase_sigma_milp = df_increase_sigma[lambda x: x.schedule_type == 'milp']

In [ ]:
_, ax = plt.subplots(figsize=np.array((4.5,2))*1, dpi=100)

df_filtered = df_increase_sigma_milp.drop_duplicates(['sigma'])
X = df_filtered.sigma
Y = df_filtered.execution_time
baseline_execution_time = df_increase_sigma[lambda x: x.schedule_type == 'naive'].iloc[0].execution_time

plt.bar(X, Y, linewidth=.8, edgecolor='black', color=box_color, width=0.7)
xmin = 0.1
xmax = 20.8
ax.set_xlim([xmin, xmax])
plt.axhline(baseline_execution_time, color='red')
plt.text(xmax-0.1, baseline_execution_time+100, "Naive strategy", c='red', ha='right', va='bottom', fontsize=9)

ax.set_axisbelow(False)
plt.grid(axis='y')
plt.xticks(X, fontsize=8)
plt.xlabel("$\sigma$")
plt.ylabel("Execution time (s)")

plt.savefig("../out/figures/1_planning_horizon_increase_sigma.pdf", bbox_inches='tight')
plt.show()

In [ ]:
# Calculate precise gain

In [ ]:
lowest_execution_time = df_increase_sigma[lambda x: x.execution_time < baseline_execution_time].execution_time.min()
print(f"Lowest execution time = {lowest_execution_time:.1f}s")
print(f"                   or = {(1 - (lowest_execution_time / baseline_execution_time)) * 100:.1f}% faster than naive")

In [ ]:
highest_execution_time = df_increase_sigma[lambda x: x.execution_time < baseline_execution_time].execution_time.max()
print(f"Highest execution time = {highest_execution_time:.1f}s")
print(f"                    or = {(1 - (highest_execution_time / baseline_execution_time)) * 100:.1f}% faster than naive")

## Full coverage

In [ ]:
basedir = "../out/villalvernia/1_planning_horizon/full_coverage"
df_full_coverage = load_data(basedir)
df_full_coverage_milp = df_full_coverage[lambda x: x.schedule_type == 'milp']
df_full_coverage_milp = df_full_coverage_milp[lambda x: x.W > 6]

In [ ]:
# Execution times

In [ ]:
X = df_full_coverage_milp.W
Y = df_full_coverage_milp.execution_time
baseline_execution_time = df_full_coverage[lambda x: x.schedule_type == 'naive'].iloc[0].execution_time

_, ax = plt.subplots(figsize=np.array((4.5,2))*1, dpi=100)
plt.bar(X, Y, linewidth=.8, edgecolor='black', color=box_color, width=0.7)

xmin = 6.1
xmax = 20.8
ax.set_xlim([xmin, xmax])
plt.axhline(baseline_execution_time, color='red')
plt.text(xmax-0.1, baseline_execution_time-50, "Naive strategy", c='red', ha='right', va='top', fontsize=9)

plt.grid(axis='y')
ax.set_xticks(X)
ax.set_xlabel("W")
ax.set_ylabel("Execution time (s)")

plt.savefig("../out/figures/1_planning_horizon_full_coverage_execution_time.pdf", bbox_inches='tight')
plt.show()

In [ ]:
lowest_execution_time = df_full_coverage_milp.execution_time.min()
print(f"Lowest execution time = {lowest_execution_time:.1f}s")
print(f"                   or = {(1 - (lowest_execution_time / baseline_execution_time)) * 100:.1f}% faster than naive")

highest_execution_time = df_full_coverage_milp.execution_time.max()
print(f"Highest execution time = {highest_execution_time:.1f}s")
print(f"                    or = {(1 - (highest_execution_time / baseline_execution_time)) * 100:.1f}% faster than naive")

In [ ]:
# Solve times (total)

In [ ]:
X = sorted(df_full_coverage_milp.W.unique())
Y = df_full_coverage_milp.groupby('W').solve_time.sum()

_, ax = plt.subplots(figsize=np.array((4.5,2))*1, dpi=100)
plt.bar(X, Y, linewidth=.8, edgecolor='black', color=box_color, width=0.7)

plt.grid(axis='y')
xmin = 6.1
xmax = 20.8
ax.set_xlim([xmin, xmax])
ax.set_xticks(X)
ax.set_xlabel("W")
ax.set_ylabel("Total solve time (s)")

plt.savefig("../out/figures/1_planning_horizon_full_coverage_solve_time_total.pdf", bbox_inches='tight')
plt.show()

In [ ]:
# Solve times (boxplot)

In [ ]:
_, ax = plt.subplots(figsize=np.array((4.5,2))*1, dpi=100)
sns.boxplot(x="W", y="solve_time", data=df_full_coverage_milp, ax=ax, width=0.6, linewidth=1, whis=[0, 100], zorder=10, color=box_color)
plt.yscale('log')
plt.ylabel("Solve time [log] (s)")

ticks = [1, 10, 60, 600, 1800]
labels = ['1s', '10s', '1m', '10m', '30m']
plt.yticks(ticks, labels)
ax.set_axisbelow(True)
plt.grid(axis='y')

plt.savefig("../out/figures/1_planning_horizon_full_coverage_solve_time_box.pdf", bbox_inches='tight')
plt.show()

# Number of drones

In [ ]:
basedir = "../out/villalvernia/2_n_drones"
df_n_drones = load_data(basedir)
df_n_drones_milp = df_n_drones[lambda x: (x.schedule_type == 'milp') & (x.W == 20)].drop_duplicates(['schedule_type', 'n_drones'])
df_n_drones_naive = df_n_drones[lambda x: x.schedule_type == 'naive'].drop_duplicates(['schedule_type', 'n_drones'])

In [ ]:
_, ax = plt.subplots(figsize=np.array((4.5,2))*1, dpi=100)
df_plot = pd.concat([df_n_drones_milp, df_n_drones_naive])
sns.barplot(x='n_drones', y='execution_time', hue='schedule_type', data=df_plot, edgecolor='black', linewidth=0.75)

# add gains
for i, n_drones in enumerate(sorted(df_plot.n_drones.unique())):
    milp_time = df_n_drones_milp[lambda x: x.n_drones == n_drones].iloc[0].execution_time
    naive_time = df_n_drones_naive[lambda x: x.n_drones == n_drones].iloc[0].execution_time
    max_time = max(milp_time, naive_time)
    rel_perc = (milp_time / naive_time * 100)
    diff = 100 - rel_perc

    if diff > 0:
        color = 'green'
        txt = r"$\downarrow$" + f"{-diff:.1f}%"
    else:
        color = 'red'
        txt = r"$\uparrow$" + f"+{-diff:.1f}%"
    plt.text(i, max_time + 50, txt, ha='center', va='bottom', backgroundcolor='white', color=color, fontsize=9, bbox=dict(boxstyle='square,pad=0', fc='white', ec='none'))
    
ymin, ymax = plt.ylim()
plt.ylim([ymin, ymax+500])
plt.legend(fontsize=8)
plt.xlabel("$N_d$")
plt.ylabel("Execution time (s)")

ax.set_axisbelow(True)
plt.grid(axis='y')

plt.savefig("../out/figures/2_n_drones.pdf", bbox_inches='tight')
plt.show()